In [36]:
import requests
import json
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
from pymongo import MongoClient
dbName = "APIchat"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL)
# Conectar la db
db = client.get_database()
db = client.get_default_database()["comments"]
print(db)

mongodb://localhost/APIchat
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'APIchat'), 'comments')


## Extraer todos los mensajes y usuario

In [41]:
mensajes = list(db.find({"type": "message"}, {"user_id":1, "text":1, "_id":0}))
print(mensajes)

[{'user_id': ObjectId('5ead91e4877d9a546966a8f6'), 'text': "I was so sure this was American. Disappointed to realize it's the UK. Great example it will be getting arrested in front of your kids. Thoroughly deserve it though"}, {'user_id': ObjectId('5ead91da877d9a546966a8f5'), 'text': 'These parents should be made to stand in the naughty corner and be grounded with no pocket money for the next month!'}, {'user_id': ObjectId('5ead9196877d9a546966a8f1'), 'text': "Why do I have a feeling at some point in the near future these 'protesters' will experience the same as the protesters in the US; and then discover the harsh reality of their stupidness of refusing to follow the simplest of lockdown requests?!"}, {'user_id': ObjectId('5ead91cb877d9a546966a8f4'), 'text': 'Perhaps the media should be giving a more consistent message. Too many articles claiming lockdown isn’t working or we should start getting back to normal and not enough articles demonstrating the benefits of lockdown and the impo

In [18]:
usuarios = [e["user_id"] for e in mensajes]
textos = [e["text"] for e in mensajes]

## Extraer los comentarios a un data frame

In [27]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/lee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lee/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [60]:
def textCleaner(frase):
    stop_words = set(stopwords.words('english')) 
    # Tokeniza
    word_tokens = word_tokenize(frase)
    # Extract whatever is not a stop word
    cleaned = [w for w in word_tokens if w not in stop_words]
    joined = " ".join(cleaned)
    return joined

In [61]:
prueba = []
for e in textos:
    prueba.append(textCleaner(e))
print(prueba)

["I sure American . Disappointed realize 's UK . Great example getting arrested front kids . Thoroughly deserve though", 'These parents made stand naughty corner grounded pocket money next month !', "Why I feeling point near future 'protesters ' experience protesters US ; discover harsh reality stupidness refusing follow simplest lockdown requests ? !", 'Perhaps media giving consistent message . Too many articles claiming lockdown ’ working start getting back normal enough articles demonstrating benefits lockdown importance abiding guidance .', "Because many people got 'Locked ' brains think Lockdown NEVER end news pumping heads day day 2 months Lockdown Stay Home Social Distancing", 'Rats breeding rats', 'Brainless chicken .', 'Well get virus dont treat . Serves right .', 'As government forking millions pounds money pay people sit backsides fun , rest us put lives risk working whatever field keep country going hell . Why earth would want lie figures . It serves purpose . Just cause lo

In [64]:
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(prueba)
print(list(count_vectorizer.vocabulary_.keys()))
m = sparse_matrix.todense()
print(m.shape)
print(m[0])

['sure', 'american', 'disappointed', 'realize', 'uk', 'great', 'example', 'getting', 'arrested', 'front', 'kids', 'thoroughly', 'deserve', 'though', 'these', 'parents', 'made', 'stand', 'naughty', 'corner', 'grounded', 'pocket', 'money', 'next', 'month', 'why', 'feeling', 'point', 'near', 'future', 'protesters', 'experience', 'us', 'discover', 'harsh', 'reality', 'stupidness', 'refusing', 'follow', 'simplest', 'lockdown', 'requests', 'perhaps', 'media', 'giving', 'consistent', 'message', 'too', 'many', 'articles', 'claiming', 'working', 'start', 'back', 'normal', 'enough', 'demonstrating', 'benefits', 'importance', 'abiding', 'guidance', 'because', 'people', 'got', 'locked', 'brains', 'think', 'never', 'end', 'news', 'pumping', 'heads', 'day', 'months', 'stay', 'home', 'social', 'distancing', 'rats', 'breeding', 'brainless', 'chicken', 'well', 'get', 'virus', 'dont', 'treat', 'serves', 'right', 'as', 'government', 'forking', 'millions', 'pounds', 'pay', 'sit', 'backsides', 'fun', 'rest

In [65]:
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=usuarios)
display(df)

,19,1st,abiding,able,absorption,advise,all,although,american,animal,...,whilst,why,win,without,working,would,write,wrong,you,young
5ead91e4877d9a546966a8f6,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5ead91da877d9a546966a8f5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead9196877d9a546966a8f1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5ead91cb877d9a546966a8f4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5ead9179877d9a546966a8ef,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead91c2877d9a546966a8f3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead9171877d9a546966a8ee,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead91b6877d9a546966a8f2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead91b6877d9a546966a8f2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
5ead9138877d9a546966a8ed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
from sklearn.metrics.pairwise import cosine_similarity as distance
#from sklearn.metrics.pairwise import euclidean_distances as distance
similarity_matrix = distance(df,df)
print(similarity_matrix)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.35355339]
 [0.         0.         0.         ... 0.         0.35355339 1.        ]]
